<a href="https://colab.research.google.com/github/AliceInHunterland/colab/blob/main/SpeechSplit_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numba==0.48
!pip install librosa==0.6.3

     |████████████████████████████████| 3.5MB 13.8MB/s 
     |████████████████████████████████| 20.2MB 1.3MB/s 
ERROR: umap-learn 0.5.0 has requirement numba>=0.49, but you'll have numba 0.48.0 which is incompatible.
ERROR: pynndescent 0.5.1 has requirement numba>=0.51.2, but you'll have numba 0.48.0 which is incompatible.
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
     |████████████████████████████████| 1.6MB 13.6MB/s 
  Created wheel for librosa: filename=librosa-0.6.3-cp36-none-any.whl size=1573317 sha256=46bb89f92d7bf584fa9831a26ca77d0a27a890a8c4c6c6321234347b3d4ce2b4
  Stored in directory: /root/.cache/pip/wheels/ce/49/68/87ba660f30e3977f0778e39ee2e944629cd37c2a0ce41f9ff1
Successfully built librosa
  Found existing installation: librosa 0.8.0
    Uninstalling librosa-0.8.0:
      

In [3]:
import librosa
print(librosa.__version__) #should be 0.6.3

0.6.3


In [4]:
!git clone https://github.com/auspicious3000/SpeechSplit.git

Cloning into 'SpeechSplit'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 37 (delta 12), reused 34 (delta 9), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [5]:
cd SpeechSplit/

/content/SpeechSplit


In [6]:
cd assets/

/content/SpeechSplit/assets


In [7]:
!gdown --id {"1JF1WNS57wWcbmn1EztJxh09xU739j4_g"} -O model.zip

Downloading...
From: https://drive.google.com/uc?id=1JF1WNS57wWcbmn1EztJxh09xU739j4_g
To: /content/SpeechSplit/assets/model.zip
253MB [00:02, 85.1MB/s]


In [8]:
!gdown --id 1Zksy0ndlDezo9wclQNZYkGi_6i7zi4nQ -O checkpoint_step001000000_ema.pth

Downloading...
From: https://drive.google.com/uc?id=1Zksy0ndlDezo9wclQNZYkGi_6i7zi4nQ
To: /content/SpeechSplit/assets/checkpoint_step001000000_ema.pth
297MB [00:04, 60.9MB/s]


In [9]:
!gdown --id {"1SZPPnWAgpGrh0gQ7bXQJXXjOntbh4hmz"} -O autovc.ckpt

Downloading...
From: https://drive.google.com/uc?id=1SZPPnWAgpGrh0gQ7bXQJXXjOntbh4hmz
To: /content/SpeechSplit/assets/autovc.ckpt
341MB [00:04, 68.5MB/s]


In [10]:
!unzip model.zip

Archive:  model.zip
  inflating: 640000-P.ckpt           
  inflating: 660000-G.ckpt           


In [11]:
pip install wavenet_vocoder==0.1.1

  Created wheel for wavenet-vocoder: filename=wavenet_vocoder-0.1.1-cp36-none-any.whl size=12668 sha256=716c67bf9f29a31fcf82a21125a02b6d5fa8e311c510101f66e706e7f7eacbab
  Stored in directory: /root/.cache/pip/wheels/72/fc/21/02d3785b65dd072b110b44b9df98b8cbf72a89ddea424ff0d9
Successfully built wavenet-vocoder


In [12]:
!git clone https://github.com/auspicious3000/autovc.git

Cloning into 'autovc'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 95 (delta 10), reused 60 (delta 3), pack-reused 23
Unpacking objects: 100% (95/95), done.


In [13]:
cd /content/SpeechSplit

/content/SpeechSplit


In [14]:
# demo conversion
import torch
import pickle
import numpy as np
from hparams import hparams
from utils import pad_seq_to_2
from utils import quantize_f0_numpy
from model import Generator_3 as Generator
from model import Generator_6 as F0_Converter


device = 'cuda:0'
G = Generator(hparams).eval().to(device)
g_checkpoint = torch.load('assets/660000-G.ckpt', map_location=lambda storage, loc: storage)
G.load_state_dict(g_checkpoint['model'])

P = F0_Converter(hparams).eval().to(device)
p_checkpoint = torch.load('assets/640000-P.ckpt', map_location=lambda storage, loc: storage)
P.load_state_dict(p_checkpoint['model'])


metadata = pickle.load(open('assets/demo.pkl', "rb"))


sbmt_i = metadata[0]
emb_org = torch.from_numpy(sbmt_i[1]).to(device)
x_org, f0_org, len_org, uid_org = sbmt_i[2]        
uttr_org_pad, len_org_pad = pad_seq_to_2(x_org[np.newaxis,:,:], 192)
uttr_org_pad = torch.from_numpy(uttr_org_pad).to(device)
f0_org_pad = np.pad(f0_org, (0, 192-len_org), 'constant', constant_values=(0, 0))
f0_org_quantized = quantize_f0_numpy(f0_org_pad)[0]
f0_org_onehot = f0_org_quantized[np.newaxis, :, :]
f0_org_onehot = torch.from_numpy(f0_org_onehot).to(device)
uttr_f0_org = torch.cat((uttr_org_pad, f0_org_onehot), dim=-1)

sbmt_j = metadata[1]
emb_trg = torch.from_numpy(sbmt_j[1]).to(device)
x_trg, f0_trg, len_trg, uid_trg = sbmt_j[2]        
uttr_trg_pad, len_trg_pad = pad_seq_to_2(x_trg[np.newaxis,:,:], 192)
uttr_trg_pad = torch.from_numpy(uttr_trg_pad).to(device)
f0_trg_pad = np.pad(f0_trg, (0, 192-len_trg), 'constant', constant_values=(0, 0))
f0_trg_quantized = quantize_f0_numpy(f0_trg_pad)[0]
f0_trg_onehot = f0_trg_quantized[np.newaxis, :, :]
f0_trg_onehot = torch.from_numpy(f0_trg_onehot).to(device)

with torch.no_grad():
    f0_pred = P(uttr_org_pad, f0_trg_onehot)[0]
    f0_pred_quantized = f0_pred.argmax(dim=-1).squeeze(0)
    f0_con_onehot = torch.zeros((1, 192, 257), device=device)
    f0_con_onehot[0, torch.arange(192), f0_pred_quantized] = 1
uttr_f0_trg = torch.cat((uttr_org_pad, f0_con_onehot), dim=-1)    


conditions = ['R', 'F', 'U', 'RF', 'RU', 'FU', 'RFU']
spect_vc = []
with torch.no_grad():
    for condition in conditions:
        if condition == 'R':
            x_identic_val = G(uttr_f0_org, uttr_trg_pad, emb_org)
        if condition == 'F':
            x_identic_val = G(uttr_f0_trg, uttr_org_pad, emb_org)
        if condition == 'U':
            x_identic_val = G(uttr_f0_org, uttr_org_pad, emb_trg)
        if condition == 'RF':
            x_identic_val = G(uttr_f0_trg, uttr_trg_pad, emb_org)
        if condition == 'RU':
            x_identic_val = G(uttr_f0_org, uttr_trg_pad, emb_trg)
        if condition == 'FU':
            x_identic_val = G(uttr_f0_trg, uttr_org_pad, emb_trg)
        if condition == 'RFU':
            x_identic_val = G(uttr_f0_trg, uttr_trg_pad, emb_trg)
            
        if 'R' in condition:
            uttr_trg = x_identic_val[0, :len_trg, :].cpu().numpy()
        else:
            uttr_trg = x_identic_val[0, :len_org, :].cpu().numpy()
                
        spect_vc.append( ('{}_{}_{}_{}'.format(sbmt_i[0], sbmt_j[0], uid_org, condition), uttr_trg ) )       

In [15]:
!touch /content/SpeechSplit/assets/autovc/__init__.py
!touch /content/SpeechSplit/assets/__init__.py
!touch /content/SpeechSplit/__init__.py
!cd /content/SpeechSplit/assets/autovc


In [16]:
cd /content/SpeechSplit/assets/autovc

/content/SpeechSplit/assets/autovc


In [17]:
ls

conversion.ipynb  LICENSE           metadata.pkl  results.pkl        wavs/
data_loader.py    main.py           model_bl.py   solver_encoder.py
hparams.py        make_metadata.py  model_vc.py   synthesis.py
__init__.py       make_spect.py     README.md     vocoder.ipynb


In [18]:
!mv hparams.py hparam.py

In [19]:
cd /content/SpeechSplit

/content/SpeechSplit


In [20]:
ls

assets/         hparams.py   main.py           model.py      solver.py
data_loader.py  __init__.py  make_metadata.py  __pycache__/  tfcompat/
demo.ipynb      LICENSE      make_spect_f0.py  README.md     utils.py


In [21]:
# coding: utf-8
"""
Synthesis waveform from trained WaveNet.

Modified from https://github.com/r9y9/wavenet_vocoder
"""

import torch
from tqdm import tqdm
import librosa
from assets.autovc.hparam import hparams
from wavenet_vocoder import builder

torch.set_num_threads(4)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


def build_model():
    
    model = getattr(builder, hparams.builder)(
        out_channels=hparams.out_channels,
        layers=hparams.layers,
        stacks=hparams.stacks,
        residual_channels=hparams.residual_channels,
        gate_channels=hparams.gate_channels,
        skip_out_channels=hparams.skip_out_channels,
        cin_channels=hparams.cin_channels,
        gin_channels=hparams.gin_channels,
        weight_normalization=hparams.weight_normalization,
        n_speakers=hparams.n_speakers,
        dropout=hparams.dropout,
        kernel_size=hparams.kernel_size,
        upsample_conditional_features=hparams.upsample_conditional_features,
        upsample_scales=hparams.upsample_scales,
        freq_axis_kernel_size=hparams.freq_axis_kernel_size,
        scalar_input=True,
        legacy=hparams.legacy,
    )
    return model



def wavegen(model, c=None, tqdm=tqdm):
    """Generate waveform samples by WaveNet.
    
    """

    model.eval()
    model.make_generation_fast_()

    Tc = c.shape[0]
    upsample_factor = hparams.hop_size
    # Overwrite length according to feature size
    length = Tc * upsample_factor

    # B x C x T
    c = torch.FloatTensor(c.T).unsqueeze(0)

    initial_input = torch.zeros(1, 1, 1).fill_(0.0)

    # Transform data to GPU
    initial_input = initial_input.to(device)
    c = None if c is None else c.to(device)

    with torch.no_grad():
        y_hat = model.incremental_forward(
            initial_input, c=c, g=None, T=length, tqdm=tqdm, softmax=True, quantize=True,
            log_scale_min=hparams.log_scale_min)

    y_hat = y_hat.view(-1).cpu().data.numpy()

    return y_hat

In [ ]:

# spectrogram to waveform
import torch
import librosa
import pickle
import os
import soundfile as sf
#from assets.autovc.synthesis import build_model, wavegen
#from  os
#from speechsplit.assets.autovc. import synthesis
#from assets.autovc import *
if not os.path.exists('results'):
    os.makedirs('results')

model = build_model().to(device)
checkpoint = torch.load("/content/SpeechSplit/assets/checkpoint_step001000000_ema.pth")
model.load_state_dict(checkpoint["state_dict"])

for spect in spect_vc:
    name = spect[0]
    c = spect[1]
    print(name)
    waveform = wavegen(model, c=c)   
    librosa.output.write_wav('results/'+name+'.wav', waveform, sr=16000)
    #sf.write('results/'+name+'.wav', waveform,16000 ,'PCM_24')

  0%|          | 5/26880 [00:00<09:48, 45.66it/s]

p226_p231_003002_R


  0%|          | 9/34560 [00:00<06:38, 86.75it/s]

p226_p231_003002_F


 66%|██████▌   | 22711/34560 [04:40<02:29, 79.17it/s]